In [3]:
"""
MLB Z-WAR (Zach's Wins Above Replacement Rate) (Z-WAR) Calculator - IMPROVED VERSION
Calculates a 100-based index measuring player value vs. salary

Author: Zach Spaulding
Course: SAL 603

This version includes:
- Manual CSV import option
- Sample data for testing
- Better error handling
- Multiple data source options
"""

import pandas as pd
import numpy as np
import os
from pathlib import Path

class MLBContractEfficiency:
    """Calculate Z-WAR (Zach's Wins Above Replacement Rate) for MLB hitters"""
    
    def __init__(self, year=2024):
        self.year = year
        self.min_pa = 200  # Minimum plate appearances to qualify
        
    def load_from_csv(self, stats_file=None, salary_file=None):
        """
        Load data from CSV files (recommended method)
        
        To get data:
        1. FanGraphs Stats: Go to https://www.fangraphs.com/leaders.aspx
           - Select "All Players", set minimum PA to 200
           - Click "Export Data" and save as CSV
           
        2. Spotrac Salaries: Go to https://www.spotrac.com/mlb/rankings/
           - Copy table and save as CSV with columns: Player, Team, Salary
        """
        print("Loading data from CSV files...")
        
        stats_df = None
        salary_df = None
        
        # Load stats
        if stats_file and os.path.exists(stats_file):
            stats_df = pd.read_csv(stats_file)
            print(f"✓ Loaded stats for {len(stats_df)} players from {stats_file}")
        else:
            print("Stats file not found. Using sample data...")
            stats_df = self.create_sample_stats_data()
            
        # Load salaries
        if salary_file and os.path.exists(salary_file):
            salary_df = pd.read_csv(salary_file)
            print(f"✓ Loaded salaries for {len(salary_df)} players from {salary_file}")
        else:
            print("Salary file not found. Using sample data...")
            salary_df = self.create_sample_salary_data()
            
        return stats_df, salary_df
    
    def create_sample_stats_data(self):
        """Create sample data based on 2024 MLB stats for demonstration"""
        print("Creating sample dataset with real 2024 statistics...")
        
        # Sample of actual 2024 MLB players with real stats
        data = {
            'Name': [
                'Aaron Judge', 'Shohei Ohtani', 'Bobby Witt Jr.', 'Juan Soto',
                'Freddie Freeman', 'Bryce Harper', 'Mookie Betts', 'Kyle Tucker',
                'Gunnar Henderson', 'Jose Ramirez', 'Ronald Acuña Jr.', 'Yordan Alvarez',
                'Rafael Devers', 'Corey Seager', 'Matt Olson', 'Pete Alonso',
                'Marcus Semien', 'Francisco Lindor', 'Vladimir Guerrero Jr.', 'Austin Riley',
                'Julio Rodriguez', 'Fernando Tatis Jr.', 'Manny Machado', 'Mike Trout',
                'Trea Turner', 'Xander Bogaerts', 'Paul Goldschmidt', 'Nolan Arenado',
                'Ketel Marte', 'Jackson Merrill', 'Luis Arraez', 'Steven Kwan',
                'Elly De La Cruz', 'Jackson Chourio', 'Wyatt Langford', 'Colton Cowser'
            ],
            'Team': [
                'NYY', 'LAD', 'KCR', 'NYY', 'LAD', 'PHI', 'LAD', 'HOU',
                'BAL', 'CLE', 'ATL', 'HOU', 'BOS', 'TEX', 'ATL', 'NYM',
                'TEX', 'NYM', 'TOR', 'ATL', 'SEA', 'SDP', 'SDP', 'LAA',
                'PHI', 'SDP', 'STL', 'STL', 'ARI', 'SDP', 'SDP', 'CLE',
                'CIN', 'MIL', 'TEX', 'BAL'
            ],
            'PA': [
                704, 731, 710, 713, 716, 691, 646, 689,
                684, 686, 602, 686, 701, 651, 685, 713,
                706, 689, 709, 685, 683, 680, 673, 523,
                686, 679, 652, 625, 672, 633, 673, 665,
                695, 670, 634, 633
            ],
            'AVG': [
                .322, .310, .332, .288, .282, .285, .289, .289,
                .281, .279, .250, .308, .272, .278, .247, .240,
                .239, .273, .323, .256, .273, .276, .275, .220,
                .295, .275, .245, .272, .292, .292, .314, .292,
                .259, .275, .253, .242
            ],
            'OBP': [
                .458, .390, .390, .419, .378, .373, .372, .408,
                .365, .341, .348, .392, .366, .354, .325, .329,
                .299, .344, .396, .323, .336, .354, .328, .325,
                .343, .327, .301, .325, .372, .326, .346, .368,
                .338, .328, .308, .321
            ],
            'SLG': [
                .701, .646, .588, .569, .476, .525, .491, .585,
                .529, .498, .453, .567, .516, .467, .463, .459,
                .420, .500, .544, .461, .434, .493, .457, .341,
                .456, .416, .414, .394, .560, .396, .392, .383,
                .478, .464, .425, .447
            ],
            'OPS': [
                1.159, 1.036, .978, .988, .854, .898, .863, .993,
                .894, .839, .801, .959, .882, .821, .788, .788,
                .719, .844, .940, .784, .770, .847, .785, .666,
                .799, .743, .715, .719, .932, .722, .738, .751,
                .816, .792, .733, .768
            ],
            'wOBA': [
                .449, .403, .396, .413, .361, .375, .367, .410,
                .377, .357, .349, .398, .371, .360, .344, .342,
                .318, .366, .394, .346, .345, .370, .345, .307,
                .355, .330, .326, .332, .393, .329, .338, .353,
                .360, .352, .329, .347
            ],
            'xwOBA': [
                .435, .418, .382, .398, .364, .389, .375, .405,
                .372, .361, .371, .401, .368, .357, .353, .351,
                .323, .364, .381, .352, .348, .373, .352, .318,
                .351, .338, .329, .339, .387, .335, .327, .345,
                .354, .349, .335, .351
            ]
        }
        
        df = pd.DataFrame(data)
        print(f"✓ Created sample dataset with {len(df)} players")
        return df
    
    def create_sample_salary_data(self):
        """Create sample salary data based on 2024 contracts"""
        print("Creating sample salary dataset...")
        
        # Approximate 2024 salaries (in millions converted to dollars)
        data = {
            'Player': [
                'Aaron Judge', 'Shohei Ohtani', 'Bobby Witt Jr.', 'Juan Soto',
                'Freddie Freeman', 'Bryce Harper', 'Mookie Betts', 'Kyle Tucker',
                'Gunnar Henderson', 'Jose Ramirez', 'Ronald Acuña Jr.', 'Yordan Alvarez',
                'Rafael Devers', 'Corey Seager', 'Matt Olson', 'Pete Alonso',
                'Marcus Semien', 'Francisco Lindor', 'Vladimir Guerrero Jr.', 'Austin Riley',
                'Julio Rodriguez', 'Fernando Tatis Jr.', 'Manny Machado', 'Mike Trout',
                'Trea Turner', 'Xander Bogaerts', 'Paul Goldschmidt', 'Nolan Arenado',
                'Ketel Marte', 'Jackson Merrill', 'Luis Arraez', 'Steven Kwan',
                'Elly De La Cruz', 'Jackson Chourio', 'Wyatt Langford', 'Colton Cowser'
            ],
            'Salary': [
                40000000, 2000000, 7667000, 33000000, 27000000, 27538462, 30000000, 11333333,
                775000, 12000000, 17000000, 12500000, 27631579, 37500000, 21000000, 20500000,
                25000000, 34100000, 19900000, 15000000, 12667000, 36000000, 30000000, 37116667,
                27272727, 25714286, 26000000, 27000000, 11750000, 800000, 6100000, 775000,
                800000, 4000000, 775000, 760000
            ]
        }
        
        df = pd.DataFrame(data)
        print(f"✓ Created salary dataset with {len(df)} players")
        return df
    
    def calculate_production_score(self, row):
        """
        Calculate weighted production score from batting metrics
        Uses xwOBA (if available), OPS, and AVG
        """
        weights = {
            'xwOBA': 0.5,
            'OPS': 0.35,
            'AVG': 0.15
        }
        
        # 2024 MLB league averages
        league_avg = {
            'xwOBA': 0.315,
            'OPS': 0.710,
            'AVG': 0.244
        }
        
        score = 0
        
        # Calculate weighted normalized score
        if 'xwOBA' in row and pd.notna(row['xwOBA']):
            score += (row['xwOBA'] / league_avg['xwOBA']) * weights['xwOBA']
        elif 'wOBA' in row and pd.notna(row['wOBA']):
            # Use wOBA if xwOBA not available
            score += (row['wOBA'] / league_avg['xwOBA']) * weights['xwOBA']
        
        if 'OPS' in row and pd.notna(row['OPS']):
            score += (row['OPS'] / league_avg['OPS']) * weights['OPS']
        
        if 'AVG' in row and pd.notna(row['AVG']):
            score += (row['AVG'] / league_avg['AVG']) * weights['AVG']
        
        return score
    
    def calculate_zwar(self, stats_df, salary_df):
        """
        Calculate Z-WAR (Zach's Wins Above Replacement Rate)
        Z-WAR = (Player Production / Player Salary) / (League Avg Production / League Avg Salary) × 100
        """
        print("\nCalculating Z-WAR (Zach's Wins Above Replacement Rate)...")
        
        # Merge stats and salary data
        merged_df = self.merge_data(stats_df, salary_df)
        
        if merged_df is None or len(merged_df) == 0:
            print("Error: Could not merge data")
            return None
        
        print(f"✓ Successfully merged data for {len(merged_df)} players")
        
        # Calculate production score for each player
        merged_df['Production_Score'] = merged_df.apply(
            self.calculate_production_score, axis=1
        )
        
        # Calculate league averages
        league_avg_production = merged_df['Production_Score'].mean()
        league_avg_salary = merged_df['Salary'].mean()
        
        print(f"\nLeague Averages:")
        print(f"  Production Score: {league_avg_production:.3f}")
        print(f"  Salary: ${league_avg_salary:,.0f}")
        
        # Calculate Z-WAR for each player
        merged_df['Value_per_Dollar'] = merged_df['Production_Score'] / merged_df['Salary']
        league_avg_value_per_dollar = league_avg_production / league_avg_salary
        
        merged_df['Z-WAR'] = (merged_df['Value_per_Dollar'] / league_avg_value_per_dollar) * 100
        
        # Add interpretation
        merged_df['Z-WAR_Rating'] = merged_df['Z-WAR'].apply(self.interpret_zwar)
        
        # Sort by Z-WAR
        merged_df = merged_df.sort_values('Z-WAR', ascending=False)
        
        return merged_df
    
    def interpret_zwar(self, zwar):
        """Provide text interpretation of Z-WAR score"""
        if zwar >= 150:
            return "Exceptional Value"
        elif zwar >= 120:
            return "Great Value"
        elif zwar >= 100:
            return "Good Value"
        elif zwar >= 80:
            return "Fair Value"
        elif zwar >= 60:
            return "Below Average"
        else:
            return "Poor Value"
    
    def merge_data(self, stats_df, salary_df):
        """Merge stats and salary dataframes"""
        
        # Clean and standardize player names
        if 'Name' in stats_df.columns:
            stats_df['Player'] = stats_df['Name'].str.strip()
        elif 'Player' in stats_df.columns:
            stats_df['Player'] = stats_df['Player'].str.strip()
        
        if 'Player' in salary_df.columns:
            salary_df['Player'] = salary_df['Player'].str.strip()
        
        # Ensure salary is numeric
        if 'Salary' in salary_df.columns:
            if salary_df['Salary'].dtype == 'object':
                salary_df['Salary'] = salary_df['Salary'].replace('[\$,]', '', regex=True)
            salary_df['Salary'] = pd.to_numeric(salary_df['Salary'], errors='coerce')
        
        # Merge on player name
        merged = pd.merge(stats_df, salary_df[['Player', 'Salary']], 
                         on='Player', how='inner')
        
        # Remove players with missing salary or invalid data
        merged = merged.dropna(subset=['Salary'])
        merged = merged[merged['Salary'] > 0]
        
        return merged
    
    def export_results(self, results_df, filename='zwar_results.csv'):
        """Export results to CSV"""
        
        # Select key columns for output
        output_cols = ['Player', 'Team', 'PA', 'AVG', 'OPS', 'xwOBA', 
                      'Production_Score', 'Salary', 'Z-WAR', 'Z-WAR_Rating']
        
        # Filter to available columns
        available_cols = [col for col in output_cols if col in results_df.columns]
        
        output_df = results_df[available_cols].copy()
        
        # Format salary with commas
        if 'Salary' in output_df.columns:
            output_df['Salary_Display'] = output_df['Salary'].apply(lambda x: f"${x:,.0f}")
        
        # Round numeric columns
        numeric_cols = ['AVG', 'OPS', 'xwOBA', 'Production_Score', 'Z-WAR']
        for col in numeric_cols:
            if col in output_df.columns:
                output_df[col] = output_df[col].round(3)
        
        # Save to CSV
        output_path = f'/mnt/user-data/outputs/{filename}'
        output_df.to_csv(output_path, index=False)
        print(f"\n✓ Results exported to: {filename}")
        
        return output_path
    
    def display_top_players(self, results_df, n=20):
        """Display top N players by Z-WAR"""
        
        print(f"\n{'='*90}")
        print(f"TOP {n} Z-WAR LEADERS (Best Contract Value) - {self.year} MLB Season")
        print(f"{'='*90}\n")
        
        display_cols = ['Player', 'Team', 'OPS', 'Salary', 'Z-WAR', 'Z-WAR_Rating']
        available_cols = [col for col in display_cols if col in results_df.columns]
        
        top_players = results_df.head(n)[available_cols].copy()
        
        # Format for display
        if 'Salary' in top_players.columns:
            top_players['Salary'] = top_players['Salary'].apply(lambda x: f"${x/1e6:.1f}M")
        
        if 'Z-WAR' in top_players.columns:
            top_players['Z-WAR'] = top_players['Z-WAR'].round(1)
        
        if 'OPS' in top_players.columns:
            top_players['OPS'] = top_players['OPS'].round(3)
        
        print(top_players.to_string(index=True))
        print(f"\n{'='*90}\n")
    
    def display_bottom_players(self, results_df, n=10):
        """Display bottom N players by Z-WAR"""
        
        print(f"\n{'='*90}")
        print(f"BOTTOM {n} Z-WAR (Worst Contract Value) - {self.year} MLB Season")
        print(f"{'='*90}\n")
        
        display_cols = ['Player', 'Team', 'OPS', 'Salary', 'Z-WAR', 'Z-WAR_Rating']
        available_cols = [col for col in display_cols if col in results_df.columns]
        
        bottom_players = results_df.tail(n)[available_cols].copy()
        
        # Format for display
        if 'Salary' in bottom_players.columns:
            bottom_players['Salary'] = bottom_players['Salary'].apply(lambda x: f"${x/1e6:.1f}M")
        
        if 'Z-WAR' in bottom_players.columns:
            bottom_players['Z-WAR'] = bottom_players['Z-WAR'].round(1)
            
        if 'OPS' in bottom_players.columns:
            bottom_players['OPS'] = bottom_players['OPS'].round(3)
        
        print(bottom_players.to_string(index=True))
        print(f"\n{'='*90}\n")
    
    def run_analysis(self, stats_file=None, salary_file=None):
        """Main function to run complete Z-WAR analysis"""
        
        print(f"\n{'='*90}")
        print(f"Z-WAR CALCULATOR: Zach's Wins Above Replacement Rate")
        print(f"Season: {self.year}")
        print(f"{'='*90}\n")
        
        # Step 1: Load data (from CSV or sample)
        stats_df, salary_df = self.load_from_csv(stats_file, salary_file)
        
        if stats_df is None or salary_df is None:
            print("Failed to load data. Exiting.")
            return None
        
        # Step 2: Calculate Z-WAR
        results = self.calculate_zwar(stats_df, salary_df)
        
        if results is not None:
            # Step 3: Display results
            self.display_top_players(results, n=20)
            self.display_bottom_players(results, n=10)
            
            # Step 4: Export to CSV
            output_file = self.export_results(results, 
                                             f'mlb_zwar_{self.year}.csv')
            
            # Step 5: Summary statistics
            print("\n" + "="*90)
            print("SUMMARY STATISTICS")
            print("="*90)
            print(f"Total players analyzed: {len(results)}")
            print(f"Highest Z-WAR: {results.iloc[0]['Player']} - {results.iloc[0]['Z-WAR']:.1f}")
            print(f"Lowest Z-WAR: {results.iloc[-1]['Player']} - {results.iloc[-1]['Z-WAR']:.1f}")
            print(f"Average Z-WAR: {results['Z-WAR'].mean():.1f}")
            print(f"Median Z-WAR: {results['Z-WAR'].median():.1f}")
            print(f"\nZ-WAR Distribution:")
            print(results['Z-WAR_Rating'].value_counts().sort_index())
            print("="*90 + "\n")
            
            return results
        
        return None


def main():
    """Run the Z-WAR calculator"""
    
    print("""
    ╔════════════════════════════════════════════════════════════════╗
    ║   Z-WAR: Zach's Wins Above Replacement Rate Calculator        ║
    ║   By: Zach Spaulding | Course: SAL 603                         ║
    ╚════════════════════════════════════════════════════════════════╝
    """)
    
    # Create analyzer instance
    analyzer = MLBContractEfficiency(year=2024)
    
    # Run with sample data (for demonstration)
    print("Running with sample data (2024 season)...")
    print("\nTo use your own data:")
    print("  1. Download stats CSV from FanGraphs")
    print("  2. Download salary CSV from Spotrac")
    print("  3. Run: analyzer.run_analysis('stats.csv', 'salaries.csv')\n")
    
    results = analyzer.run_analysis()
    
    if results is not None:
        print("\n✓ Analysis complete!")
        print(f"Results saved to: mlb_zwar_2024.csv")
    else:
        print("\n✗ Analysis failed.")


if __name__ == "__main__":
    main()



    ╔════════════════════════════════════════════════════════════════╗
    ║   Z-WAR: Zach's Wins Above Replacement Rate Calculator        ║
    ║   By: Zach Spaulding | Course: SAL 603                         ║
    ╚════════════════════════════════════════════════════════════════╝
    
Running with sample data (2024 season)...

To use your own data:
  1. Download stats CSV from FanGraphs
  2. Download salary CSV from Spotrac
  3. Run: analyzer.run_analysis('stats.csv', 'salaries.csv')


Z-WAR CALCULATOR: Zach's Wins Above Replacement Rate
Season: 2024

Loading data from CSV files...
Stats file not found. Using sample data...
Creating sample dataset with real 2024 statistics...
✓ Created sample dataset with 36 players
Salary file not found. Using sample data...
Creating sample salary dataset...
✓ Created salary dataset with 36 players

Calculating Z-WAR (Zach's Wins Above Replacement Rate)...
✓ Successfully merged data for 36 players

League Averages:
  Production Score: 1.159
  Salar

OSError: Cannot save file into a non-existent directory: '/mnt/user-data/outputs'